In [1]:
import requests
import sqlite3
!pip install requests
import pandas as pd

In [2]:
#define the api
products_api = "https://fakestoreapi.com/products"
carts_api = "https://fakestoreapi.com/carts"
users_api = "https://fakestoreapi.com/users"

# fetching the data

In [3]:
# fetching product data
products_api = "https://fakestoreapi.com/products"
# making a get request
products_response = requests.get(products_api)
# verrify if successful
if products_response.status_code==200:
    products_data = products_response.json()# parse the json response
else:
    print(f"failed to fetch data.status_code: {response.status_code}")

In [4]:
print(products_data[0])

{'id': 1, 'title': 'Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops', 'price': 109.95, 'description': 'Your perfect pack for everyday use and walks in the forest. Stash your laptop (up to 15 inches) in the padded sleeve, your everyday', 'category': "men's clothing", 'image': 'https://fakestoreapi.com/img/81fPKd-2AYL._AC_SL1500_.jpg', 'rating': {'rate': 3.9, 'count': 120}}


In [5]:
# fetching the carts data
carts_api = "https://fakestoreapi.com/carts"
# making a get request
carts_response = requests.get(carts_api)
# verify if successful
if carts_response.status_code==200:
    carts_data=carts_response.json()# parse the json response
else:
    print(f"failed to fetch data.status_code: {response.status_code}")

In [7]:
# fetching the users data
users_api = "https://fakestoreapi.com/users"
# making a get request
users_response = requests.get(users_api)
# verify if successful
if users_response.status_code==200:
    users_data=users_response.json()# parse the json response
else:
    print(f"failed to fetch data.status_code: {response.status_code}")

In [6]:
print(carts_data[0])

{'id': 1, 'userId': 1, 'date': '2020-03-02T00:00:00.000Z', 'products': [{'productId': 1, 'quantity': 4}, {'productId': 2, 'quantity': 1}, {'productId': 3, 'quantity': 6}], '__v': 0}


# sqlitedatase

In [9]:
conn=sqlite3.connect('fakestore.dbb')
cursor=conn.cursor()

In [8]:
print(users_data[0])

{'address': {'geolocation': {'lat': '-37.3159', 'long': '81.1496'}, 'city': 'kilcoole', 'street': 'new road', 'number': 7682, 'zipcode': '12926-3874'}, 'id': 1, 'email': 'john@gmail.com', 'username': 'johnd', 'password': 'm38rmF$', 'name': {'firstname': 'john', 'lastname': 'doe'}, 'phone': '1-570-236-7033', '__v': 0}


# create  table

In [10]:
cursor.execute("""DROP TABLE products;""")
conn.commit()

In [11]:
cursor.execute('''CREATE TABLE IF NOT EXISTS products (
                    id INTERGER,
                    title TEXT,
                    price float,
                    description TEXT,
                    category VARCHAR(255),
                    image VARCHAR(255),
                    rating_rate float,
                    rating_count INT);
                ''')
conn.commit()

In [12]:
cursor.execute('''CREATE TABLE IF NOT EXISTS carts (
                    id INTERGER,
                    userid INTERGER,
                    date TEXT);
                ''')
conn.commit()

In [13]:
cursor.execute('''CREATE TABLE IF NOT EXISTS cart_products (
                    cartid INT,
                    productId INT,
                    quantity INT,
                    FOREIGN KEY (cartId) REFERENCES carts (id))
                ''')
conn.commit()

In [14]:
cursor.execute("""DROP TABLE users;""")
conn.commit()

In [15]:
cursor.execute('''CREATE TABLE IF NOT EXISTS users (
                   geolocation_lat TEXT,
                   geolocation_long TEXT,
                   city TEXT,
                   street TEXT,
                   number INT,
                   zipcode TEXT,
                   id INT,
                   email TEXT,
                   username TEXT,
                   password TEXT,
                   firstname TEXT,
                   lastname TEXT,
                   phone TEXT)
                ''')
conn.commit()

# insert data

In [16]:
for data in products_data:
    cursor.execute(f"""INSERT INTO products (id,title,price,description,category,image,rating_rate,rating_count)
                       VALUES (?, ?, ?, ?, ?, ?, ?, ?);
                       """, (data['id'],data['title'],data['price'],data['description'],data['category'],data['image'],data['rating']['rate'],data['rating']['count']))
    conn.commit()

In [17]:
for cart in carts_data:
    cursor.execute("""INSERT OR IGNORE INTO carts (id, userId, date)
                       VALUES (:id, :userId, :date)
                       """, {'id': cart['id'],'userId':cart['userId'],'date':cart['date']})
for product in cart['products']:
    cursor.execute('''INSERT INTO cart_products (cartId, productId, quantity)
                      VALUES(:cartId, :productId, :quantity)
                   ''', {'cartId': cart['id'], 'productId':product['productId'],'quantity':product['quantity']})
    conn.commit()
   

In [19]:
for user in users_data:
    cursor.execute("""INSERT INTO users (geolocation_lat,geolocation_long,city,street,number,zipcode,id,email,username ,password,firstname,lastname,phone)
                          VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                          """, (user['address']['geolocation']['lat'],user['address']['geolocation']['long'],user['address']['city'],user['address']['street'],user['address']['number'],user['address']['zipcode'],user['id'],user['email'],user['username'],user['password'],user['name']['firstname'],user['name']['lastname'],user['phone']))
    conn.commit()

# load into database

In [ ]:
## connect to the sqlite database
conn = sqlite3.connect('fakestore.dbb')

In [21]:
products_df = pd.read_sql_query("SELECT * FROM products", conn)
products_df.head()

,id,title,price,description,category,image,rating_rate,rating_count
0,1,"Fjallraven - Foldsack No. 1 Backpack, Fits 15 ...",109.95,Your perfect pack for everyday use and walks i...,men's clothing,https://fakestoreapi.com/img/81fPKd-2AYL._AC_S...,3.9,120
1,2,Mens Casual Premium Slim Fit T-Shirts,22.30,"Slim-fitting style, contrast raglan long sleev...",men's clothing,https://fakestoreapi.com/img/71-3HjGNDUL._AC_S...,4.1,259
2,3,Mens Cotton Jacket,55.99,great outerwear jackets for Spring/Autumn/Wint...,men's clothing,https://fakestoreapi.com/img/71li-ujtlUL._AC_U...,4.7,500
3,4,Mens Casual Slim Fit,15.99,The color could be slightly different between ...,men's clothing,https://fakestoreapi.com/img/71YXzeOuslL._AC_U...,2.1,430
4,5,John Hardy Women's Legends Naga Gold & Silver ...,695.00,"From our Legends Collection, the Naga was insp...",jewelery,https://fakestoreapi.com/img/71pWzhdJNwL._AC_U...,4.6,400


In [22]:
carts_df = pd.read_sql_query("SELECT * FROM carts", conn)
carts_df.head()

cart_products_df = pd.read_sql_query("SELECT * FROM cart_products", conn)
cart_products_df.head()

,cartid,productId,quantity
0,7,18,1
1,7,18,1
2,7,18,1
3,7,18,1
4,7,18,1


In [20]:
users_df = pd.read_sql_query("SELECT * FROM users", conn)
users_df.head()

,geolocation_lat,geolocation_long,city,street,number,zipcode,id,email,username,password,firstname,lastname,phone
0,-37.3159,81.1496,kilcoole,new road,7682,12926-3874,1,john@gmail.com,johnd,m38rmF$,john,doe,1-570-236-7033
1,-37.3159,81.1496,kilcoole,Lovers Ln,7267,12926-3874,2,morrison@gmail.com,mor_2314,83r5^_,david,morrison,1-570-236-7033
2,40.3467,-30.1310,Cullman,Frances Ct,86,29567-1452,3,kevin@gmail.com,kevinryan,kev02937@,kevin,ryan,1-567-094-1345
3,50.3467,-20.1310,San Antonio,Hunters Creek Dr,6454,98234-1734,4,don@gmail.com,donero,ewedon,don,romer,1-765-789-6734
4,40.3467,-40.1310,san Antonio,adams St,245,80796-1234,5,derek@gmail.com,derek,jklg*_56,derek,powell,1-956-001-1945
